In [6]:
import dill
import re
import glob

import pandas as pd
import trueskill as ts
import numpy as np
import matplotlib.pyplot as plt

ts.setup(backend='mpmath')
RATING = {'mu': ts.MU, 'sigma': ts.SIGMA}  # initial rating

from preprocess import clean

In [67]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, Float


engine = sqlalchemy.create_engine('sqlite:///data/races.sqlite')
session = sqlalchemy.orm.sessionmaker(bind=engine)()

# Race metadata (already collected into a dataframe)

In [68]:
df_meta = pd.read_pickle('C:/data/results/df.pkl')
df_meta.head(5)

,name,date,loc,json_url,weather,strava_url,coord
race_id,,,,,,,
2,NCC 10th Tour of the Hilltowns,2008-07-26 00:00:00,,downloadrace.php?raceID=2&json=1,,http://app.strava.com/segments/,None
3,Bethel Spring Series - Ronde de Bethel,2008-03-02 00:00:00,,downloadrace.php?raceID=3&json=1,,http://app.strava.com/segments/,None
4,4th Annual Smuttynose Brewing Co. Portsmouth C...,2008-09-21 00:00:00,,downloadrace.php?raceID=4&json=1,,http://app.strava.com/segments/,None
5,Topsfield Circuit Race,2008-09-06 00:00:00,,downloadrace.php?raceID=5&json=1,,http://app.strava.com/segments/,None
6,Green Mountain Stage Crit,2008-09-01 00:00:00,,downloadrace.php?raceID=6&json=1,,http://app.strava.com/segments/,None


In [69]:
df_meta[['name', 'date', 'loc', 'json_url', 'weather']].to_sql('metadata', engine, if_exists='replace')

In [70]:

Base = declarative_base()

class Metadata(Base):
    __tablename__ = "metadata"
    
    # sqlalchemy uses these names as the column names
    race_id = Column(Integer, primary_key=True)
    name = Column(String)
    date = Column(DateTime)
    loc = Column(String)
    json_url = Column(String)
    weather = Column(String)

In [71]:
results = session.query(Metadata.race_id, Metadata.name, Metadata.date, Metadata.loc, Metadata.json_url, Metadata.weather)
for row in results[:10]:
    print(row)

(2, 'NCC 10th Tour of the Hilltowns', datetime.datetime(2008, 7, 26, 0, 0), '', 'downloadrace.php?raceID=2&json=1', '')
(3, 'Bethel Spring Series - Ronde de Bethel', datetime.datetime(2008, 3, 2, 0, 0), '', 'downloadrace.php?raceID=3&json=1', '')
(4, '4th Annual Smuttynose Brewing Co. Portsmouth Criterium', datetime.datetime(2008, 9, 21, 0, 0), '', 'downloadrace.php?raceID=4&json=1', '')
(5, 'Topsfield Circuit Race', datetime.datetime(2008, 9, 6, 0, 0), '', 'downloadrace.php?raceID=5&json=1', '')
(6, 'Green Mountain Stage Crit', datetime.datetime(2008, 9, 1, 0, 0), '', 'downloadrace.php?raceID=6&json=1', '')
(7, 'Green Mountain Stage Race Stage 2', datetime.datetime(2008, 8, 31, 0, 0), '', 'downloadrace.php?raceID=7&json=1', '')
(8, 'Green Mountain Stage Race Stage 1', datetime.datetime(2008, 8, 30, 0, 0), '', 'downloadrace.php?raceID=8&json=1', '')
(9, 'Green Mountain Stage Race Prologue', datetime.datetime(2008, 8, 29, 0, 0), '', 'downloadrace.php?raceID=9&json=1', '')
(10, 'Saco Bay


# Race results

In [84]:
index=1000
json = dill.load(open(f'C:\\data\\results\\races\\{index}.pkd', 'rb'))

df_race = pd.read_json(json)
df_race = clean(df_race).dropna(subset=['Place']) # drop DNFs
df_race = df_race.assign(race_id=int(index))
df_race.head()

,Place,RaceTime,Name,Age,Category,RacerID,TeamID,TeamName,RaceName,RaceCategoryName,IsDQ,race_id
0,1.0,,Luke Demoe,NaN,NaN,12069,13113,SLOCUM Race Team,Swan Island Rose Festival Criterium,Category 3,False,1000
1,2.0,,Joe Dengel,NaN,NaN,14499,3257,NaN,Swan Island Rose Festival Criterium,Category 3,False,1000
2,3.0,,Daniel Penner,25.0,3.0,14953,4627,Ironclad Performance Wear,Swan Island Rose Festival Criterium,Category 3,False,1000
3,4.0,,Chris Ward,NaN,NaN,10755,4513,Guinness Cycling Team,Swan Island Rose Festival Criterium,Category 3,False,1000
4,5.0,,Brian Engelhard,36.0,2.0,11409,4647,bicycleattorney.com,Swan Island Rose Festival Criterium,Category 3,False,1000


In [85]:
Base = declarative_base()

class Races(Base):
    __tablename__ = "races"
    
    # sqlalchemy uses these names as the column names
    Index = Column(Integer, primary_key=True)
    Place = Column(Integer)
    Name = Column(String)
    Age = Column(Integer)
    Category = Column(Integer)
    RacerID = Column(Integer)
    TeamID = Column(Integer)
    TeamName = Column(String)
    RaceName = Column(String)
    RaceCategoryName = Column(String)
    race_id = Column(Integer)
    prev_mu = Column(Float)
    prev_sigma = Column(Float)
    mu = Column(Float)
    sigma = Column(Float)

In [100]:
Races.__table__.drop(engine, checkfirst=True)

In [101]:
for j, (index, row) in enumerate(df_meta.iterrows()):
    if j == 100:
        break
        
    json = dill.load(open(f'C:\\data\\results\\races\\{index}.pkd', 'rb'))
        
    df_race = pd.read_json(json)
    if df_race.empty:
        continue
        
    df_race = clean(df_race).dropna(subset=['Place']) # drop DNFs
    df_race = df_race.assign(race_id=int(index), prev_mu=None, prev_sigma=None, mu=None, sigma=None)

    cols = ['Place', 'Name', 'Age', 'Category', 'RacerID', 'TeamID', 'TeamName', 'RaceName', 
            'RaceCategoryName', 'race_id', 'prev_mu', 'prev_sigma', 'mu', 'sigma']
    df_race[cols].to_sql('races', engine, if_exists='append')

In [103]:
results = session.query(Races.Index, Races.Place, Races.Name, Races.Age, Races.Category, Races.RacerID,
                        Races.TeamID, Races.TeamName, Races.RaceName, Races.RaceCategoryName, Races.race_id,
                        Races.prev_mu, Races.prev_sigma, Races.mu, Races.sigma)
for row in results[:5]:
    print(row)

(0, 1.0, 'Matt Cuttler', 29.0, 3.0, 699, 12737.0, 'CRCA/Affinity Cycles', 'NCC 10th Tour of the Hilltowns', 'Cat 4                ', 2, None, None, None, None)
(1, 2.0, 'Eric Weinrich', 42.0, 3.0, 700, 1249.0, 'portland velo-cycle-mania', 'NCC 10th Tour of the Hilltowns', 'Cat 4                ', 2, None, None, None, None)
(2, 3.0, 'Michael Boardman', 34.0, 3.0, 701, None, None, 'NCC 10th Tour of the Hilltowns', 'Cat 4                ', 2, None, None, None, None)
(3, 4.0, 'John Nobile', 45.0, 4.0, 702, None, None, 'NCC 10th Tour of the Hilltowns', 'Cat 4                ', 2, None, None, None, None)
(4, 5.0, 'Ryan Short', 20.0, 4.0, 703, 1222.0, 'TEAM PLACID PLANET', 'NCC 10th Tour of the Hilltowns', 'Cat 4                ', 2, None, None, None, None)


In [105]:
results = session.query(Races.Index, Races.Place, Races.Name, Races.RacerID, Races.RaceName, Races.RaceCategoryName, Races.race_id,
                        Races.prev_mu, Races.prev_sigma, Races.mu, Races.sigma)\
                 .group_by(Races.RaceName, Races.RaceCategoryName)
for row in results[:10]:
    print(row)

(0, 1.0, 'Bill Yabroudy', 1693, '2nd Annual CT Coast Criterium', 'Cat 3/4 Men', 43, None, None, None, None)
(14, 1.0, 'Eric Robertson', 3728, '2nd Annual CT Coast Criterium', 'Cat 4 Men', 43, None, None, None, None)
(34, 1.0, 'Nathan Etchells', 77, '2nd Annual CT Coast Criterium', 'Cat 5', 43, None, None, None, None)
(43, 1.0, 'Jeffrey Kozlowski', 717, '2nd Annual CT Coast Criterium', 'Cat 5 35+', 43, None, None, None, None)
(55, 1.0, 'Benjamin Wolfe', 74, '2nd Annual CT Coast Criterium', 'Junior Men', 43, None, None, None, None)
(57, 1.0, 'Claudio Mucci', 3651, '2nd Annual CT Coast Criterium', 'Master Men', 43, None, None, None, None)
(95, 1.0, 'Christopher Thornton', 3376, '2nd Annual CT Coast Criterium', 'Pro/1/2/3 Men', 43, None, None, None, None)
(0, 1.0, 'Marshall Johnson', 104, '4th Annual Rick Newhouse Memorial Ninigret Criterium', 'Cat 4             ', 86, None, None, None, None)
(22, 1.0, 'Greg Louro', 5739, '4th Annual Rick Newhouse Memorial Ninigret Criterium', 'Cat 5      

# Postgres

In [5]:
import dill
import re
import glob

import pandas as pd
import trueskill as ts
import numpy as np
import matplotlib.pyplot as plt

ts.setup(backend='mpmath')
RATING = {'mu': ts.MU, 'sigma': ts.SIGMA}  # initial rating

import sys
sys.path.insert(0, '..')
from preprocess import clean

In [6]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime, Float


engine = sqlalchemy.create_engine('sqlite:///data/races.sqlite')
session = sqlalchemy.orm.sessionmaker(bind=engine)()

In [42]:
df_meta = pd.read_pickle('C:/data/results/df.pkl')
df_meta.head(5)

,name,date,loc,json_url,weather,strava_url,coord
race_id,,,,,,,
2,NCC 10th Tour of the Hilltowns,2008-07-26 00:00:00,,downloadrace.php?raceID=2&json=1,,http://app.strava.com/segments/,None
3,Bethel Spring Series - Ronde de Bethel,2008-03-02 00:00:00,,downloadrace.php?raceID=3&json=1,,http://app.strava.com/segments/,None
4,4th Annual Smuttynose Brewing Co. Portsmouth C...,2008-09-21 00:00:00,,downloadrace.php?raceID=4&json=1,,http://app.strava.com/segments/,None
5,Topsfield Circuit Race,2008-09-06 00:00:00,,downloadrace.php?raceID=5&json=1,,http://app.strava.com/segments/,None
6,Green Mountain Stage Crit,2008-09-01 00:00:00,,downloadrace.php?raceID=6&json=1,,http://app.strava.com/segments/,None


In [52]:
def get_lat_lng(x):
    if x['coord']:
        x['lat'] = float(x['coord'][0])
        x['lng'] = float(x['coord'][1])
    return x
    
df_meta = df_meta.apply(get_lat_lng, axis=1)

In [53]:
df_meta

,coord,date,json_url,lat,lng,loc,name,strava_url,weather
race_id,,,,,,,,,
2,None,2008-07-26,downloadrace.php?raceID=2&json=1,NaN,NaN,,NCC 10th Tour of the Hilltowns,http://app.strava.com/segments/,
3,None,2008-03-02,downloadrace.php?raceID=3&json=1,NaN,NaN,,Bethel Spring Series - Ronde de Bethel,http://app.strava.com/segments/,
4,None,2008-09-21,downloadrace.php?raceID=4&json=1,NaN,NaN,,4th Annual Smuttynose Brewing Co. Portsmouth C...,http://app.strava.com/segments/,
5,None,2008-09-06,downloadrace.php?raceID=5&json=1,NaN,NaN,,Topsfield Circuit Race,http://app.strava.com/segments/,
6,None,2008-09-01,downloadrace.php?raceID=6&json=1,NaN,NaN,,Green Mountain Stage Crit,http://app.strava.com/segments/,
...,...,...,...,...,...,...,...,...,...
12604,"(40.4854888916016, -106.833557128906)",2020-09-06,downloadrace.php?raceID=12604&json=1,40.485489,-106.833557,"Steamboat Springs, CO",Steamboat Springs Stage Race,http://app.strava.com/segments/,"Clear, 81 degrees, wind 5 mph"
12605,"(40.4854888916016, -106.833557128906)",2020-09-06,downloadrace.php?raceID=12605&json=1,40.485489,-106.833557,"Steamboat Springs, CO",Steamboat Springs Stage Race,http://app.strava.com/segments/,"Clear, 81 degrees, wind 5 mph"
12606,"(40.7584762573242, -111.888145446777)",2020-09-26,downloadrace.php?raceID=12606&json=1,40.758476,-111.888145,"Salt Lake City, UT",East Canyon Road Race,http://app.strava.com/segments/,"Clear, 71 degrees, wind 7 mph"


In [8]:
from flask_sqlalchemy import SQLAlchemy
db = SQLAlchemy()

In [12]:
db.Float

sqlalchemy.sql.schema.Column

In [54]:
df_meta.reset_index()

,race_id,coord,date,json_url,lat,lng,loc,name,strava_url,weather
0,2,None,2008-07-26,downloadrace.php?raceID=2&json=1,NaN,NaN,,NCC 10th Tour of the Hilltowns,http://app.strava.com/segments/,
1,3,None,2008-03-02,downloadrace.php?raceID=3&json=1,NaN,NaN,,Bethel Spring Series - Ronde de Bethel,http://app.strava.com/segments/,
2,4,None,2008-09-21,downloadrace.php?raceID=4&json=1,NaN,NaN,,4th Annual Smuttynose Brewing Co. Portsmouth C...,http://app.strava.com/segments/,
3,5,None,2008-09-06,downloadrace.php?raceID=5&json=1,NaN,NaN,,Topsfield Circuit Race,http://app.strava.com/segments/,
4,6,None,2008-09-01,downloadrace.php?raceID=6&json=1,NaN,NaN,,Green Mountain Stage Crit,http://app.strava.com/segments/,
...,...,...,...,...,...,...,...,...,...,...
12127,12604,"(40.4854888916016, -106.833557128906)",2020-09-06,downloadrace.php?raceID=12604&json=1,40.485489,-106.833557,"Steamboat Springs, CO",Steamboat Springs Stage Race,http://app.strava.com/segments/,"Clear, 81 degrees, wind 5 mph"
12128,12605,"(40.4854888916016, -106.833557128906)",2020-09-06,downloadrace.php?raceID=12605&json=1,40.485489,-106.833557,"Steamboat Springs, CO",Steamboat Springs Stage Race,http://app.strava.com/segments/,"Clear, 81 degrees, wind 5 mph"
12129,12606,"(40.7584762573242, -111.888145446777)",2020-09-26,downloadrace.php?raceID=12606&json=1,40.758476,-111.888145,"Salt Lake City, UT",East Canyon Road Race,http://app.strava.com/segments/,"Clear, 71 degrees, wind 7 mph"
12130,12607,"(41.0463790893555, -95.7417907714844)",2020-09-19,downloadrace.php?raceID=12607&json=1,41.046379,-95.741791,"Glenwood, IA",2020 Glenwood Gravel Grinder,http://app.strava.com/segments/,"Clear, 67 degrees, wind 17 mph"
